In [1]:
import numpy as np
import pandas as pd
import os


path = './data/'
output = './output/'



In [2]:
df = pd.read_csv(path+'RegularSeasonDetailedResults.csv')
print(len(df))
print(list(df))

82041
['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc', 'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']


In [3]:
print(df.Season.unique())

[2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016
 2017 2018]


In [4]:
df = df.loc[df.Season>=2010]
print(len(df))

47967


In [5]:


new_cols = []
for column in list(df):
    if column == 'WLoc':
        new_cols.append('WLoc')
    else:
        column = column.replace("W","")
        column = column.replace("L","Opp")
        new_cols.append(column)
    
print(new_cols)


['Season', 'DayNum', 'TeamID', 'Score', 'OppTeamID', 'OppScore', 'WLoc', 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppOR', 'OppDR', 'OppAst', 'OppTO', 'OppStl', 'OppBlk', 'OppPF']


In [6]:
df.columns=new_cols


In [10]:
print(df.WLoc.unique())

['H' 'N' 'A']


In [9]:
opp_df = df.copy()
opp_cols = list(opp_df)

static_cols = ['Season','DayNum','WLoc','NumOT']
new_oc = []

for col in opp_cols:
    if col in static_cols:
        new_oc.append(col)
    elif "Opp" in col:
        new_oc.append(col.replace("Opp",""))
    else:
        new_oc.append('Opp'+col)

print(new_oc)

opp_df.columns=new_oc



['Season', 'DayNum', 'OppTeamID', 'OppScore', 'TeamID', 'Score', 'WLoc', 'NumOT', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppOR', 'OppDR', 'OppAst', 'OppTO', 'OppStl', 'OppBlk', 'OppPF', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF']


In [11]:
# adjust location
reverse = {
    'H':'A',
    'A':'H',
    'N':'N'
}

opp_df['WLoc'] = opp_df['WLoc'].replace(reverse)
print(opp_df.WLoc.head())

34074    A
34075    A
34076    A
34077    A
34078    A
Name: WLoc, dtype: object


In [12]:
# rearrange columns to match original
opp_df = opp_df[new_cols]
print(list(opp_df))



['Season', 'DayNum', 'TeamID', 'Score', 'OppTeamID', 'OppScore', 'WLoc', 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppOR', 'OppDR', 'OppAst', 'OppTO', 'OppStl', 'OppBlk', 'OppPF']


In [13]:
print(len(df))
df = pd.concat([df,opp_df],axis=0)
print(len(df))

df = df.rename(columns={'WLoc':'Loc'})

47967
95934


In [17]:
# possessions calc
# field goals attempted - offensive rebounds + turnovers + (0.4 x free throws attempted)

df['Poss'] = df['FGA']-df['OR']+df['TO']+(0.4*df['FTA'])
df['OppPoss'] = df['OppFGA']-df['OppOR']+df['OppTO']+(0.4*df['OppFTA'])
df['Poss'] = np.round((df['Poss']+df['OppPoss'])/2,1)
df = df.drop(columns=['OppPoss'])

df['Poss'].head()

34074    63.9
34075    82.2
34076    70.9
34077    80.0
34078    69.4
Name: Poss, dtype: float64

Desired YOY stats...
FG%,
FGM3%,
Rebound%,
Assist,Steals,Turnovers,Blocks per Poss?
Pace


In [20]:
# SeasonTeamID
df['SeaTeamID'] = df['Season'].astype(str)+df['TeamID'].astype(str)

# only need a some columns 
desired_cols = ['SeaTeamID','Season','TeamID','FGM','FGA','FGM3','FGA3','OR','DR',
                'OppFGM','OppFGA','OppFGM3','OppFGA3','OppOR','OppDR','Poss']

gb = df[desired_cols]


# first groupby Team/Season

gb = gb.groupby(['SeaTeamID'])['FGM','FGA','FGM3','FGA3','OR','DR','OppFGM','OppFGA','OppFGM3','OppFGA3','OppOR','OppDR','Poss'].mean().reset_index()
gb = gb.rename(columns={'Poss':'Pace'})

gb.head()



,SeaTeamID,FGM,FGA,FGM3,FGA3,OR,DR,OppFGM,OppFGA,OppFGM3,OppFGA3,OppOR,OppDR,Poss
0,20101102,24.000000,46.875000,7.125000,17.375000,8.250000,23.750000,19.750000,50.250000,6.250000,23.875000,9.625000,18.875000,58.975000
1,20101103,26.173913,56.347826,7.260870,19.304348,12.347826,23.869565,21.782609,54.608696,4.826087,16.304348,11.782609,20.652174,66.326087
2,20101104,26.000000,56.058824,5.529412,15.294118,12.705882,25.117647,21.764706,55.941176,5.411765,18.705882,12.529412,20.705882,64.782353
3,20101105,25.000000,59.000000,4.375000,12.250000,16.125000,26.125000,20.875000,57.125000,4.250000,17.875000,15.250000,23.250000,70.475000
4,20101106,24.076923,54.153846,6.307692,18.076923,14.307692,25.538462,21.461538,52.769231,4.692308,13.923077,9.307692,21.153846,67.492308
5,20101107,27.000000,55.714286,6.714286,17.000000,12.000000,27.714286,23.142857,58.142857,6.428571,18.857143,9.285714,19.571429,66.671429
6,20101108,23.500000,52.500000,4.500000,15.000000,9.000000,24.000000,19.000000,51.500000,4.000000,19.500000,9.500000,24.500000,69.950000
7,20101110,23.454545,49.454545,6.090909,15.636364,8.818182,27.727273,21.181818,53.636364,5.818182,19.090909,7.818182,19.909091,64.400000
8,20101111,26.150000,50.750000,7.350000,16.850000,10.250000,28.200000,23.900000,58.950000,7.650000,22.950000,10.300000,17.550000,65.415000
9,20101112,25.937500,55.062500,7.125000,18.187500,9.750000,26.687500,24.187500,59.875000,5.500000,19.437500,11.750000,22.687500,70.087500


In [22]:


gb['FG%']  = gb['FGM']/gb['FGA']
gb['FGM3%'] = gb['FGM3']/gb['FGA3']

#allowed 
gb['A_FG%']  = gb['OppFGM']/gb['OppFGA']
gb['A_FGM3%'] = gb['OppFGM3']/gb['OppFGA3']

#defensive rebound rate
gb['DRR'] = gb['DR']/(gb['DR']+gb['OppOR'])

#offensive rebound rate
gb['ORR'] = gb['OR']/(gb['OR']+gb['OppDR'])

gb = gb[['SeaTeamID','FG%','FGM3%','A_FG%','A_FGM3%','DRR','ORR','Pace']]

gb.head()

,SeaTeamID,FG%,FGM3%,A_FG%,A_FGM3%,DRR,ORR,Pace
0,20101102,0.512000,0.410072,0.393035,0.261780,0.711610,0.304147,58.975000
1,20101103,0.464506,0.376126,0.398885,0.296000,0.669512,0.374177,66.326087
2,20101104,0.463799,0.361538,0.389064,0.289308,0.667187,0.380282,64.782353
3,20101105,0.423729,0.357143,0.365427,0.237762,0.631420,0.409524,70.475000
4,20101106,0.444602,0.348936,0.406706,0.337017,0.732892,0.403471,67.492308


In [25]:
teams = df[['SeaTeamID','Season','TeamID']]
teams = teams.drop_duplicates()

gb = pd.merge(left=gb,right=teams,how='left',on=['SeaTeamID','SeaTeamID'])


In [28]:
gb['NextSeason'] = gb['Season'] + 1
gb['NextSeaTID'] = gb['NextSeason'].astype(str) + gb['TeamID'].astype(str)
gb = gb.drop(columns=['NextSeason'])
gb.head()

,SeaTeamID,FG%,FGM3%,A_FG%,A_FGM3%,DRR,ORR,Pace,Season,TeamID,NextSeason,NextSeaTID
0,20101102,0.512000,0.410072,0.393035,0.261780,0.711610,0.304147,58.975000,2010,1102,2011,20111102
1,20101103,0.464506,0.376126,0.398885,0.296000,0.669512,0.374177,66.326087,2010,1103,2011,20111103
2,20101104,0.463799,0.361538,0.389064,0.289308,0.667187,0.380282,64.782353,2010,1104,2011,20111104
3,20101105,0.423729,0.357143,0.365427,0.237762,0.631420,0.409524,70.475000,2010,1105,2011,20111105
4,20101106,0.444602,0.348936,0.406706,0.337017,0.732892,0.403471,67.492308,2010,1106,2011,20111106


In [32]:
ns = gb.copy()

ns = ns[['SeaTeamID', 'FG%','FGM3%','A_FG%','A_FGM3%','DRR','ORR','Pace']]

new_cols=[]
for col in list(ns):
    new_cols.append('N'+col)
    
ns.columns=new_cols
ns = ns.rename(columns={'NSeaTeamID':'NextSeaTID'})

gb = pd.merge(left=gb,right=ns,how='left',on=['NextSeaTID','NextSeaTID'])

gb.head()

,SeaTeamID,FG%,FGM3%,A_FG%,A_FGM3%,DRR,ORR,Pace,Season,TeamID,NextSeaTID,NFG%,NFGM3%,NA_FG%,NA_FGM3%,NDRR,NORR,NPace
0,20101102,0.512000,0.410072,0.393035,0.261780,0.711610,0.304147,58.975000,2010,1102,20111102,0.503715,0.411765,0.372796,0.333333,0.651515,0.213542,62.435714
1,20101103,0.464506,0.376126,0.398885,0.296000,0.669512,0.374177,66.326087,2010,1103,20111103,0.453517,0.387892,0.388436,0.286104,0.684274,0.287263,66.977273
2,20101104,0.463799,0.361538,0.389064,0.289308,0.667187,0.380282,64.782353,2010,1104,20111104,0.454545,0.319218,0.349565,0.271429,0.658507,0.377593,65.404762
3,20101105,0.423729,0.357143,0.365427,0.237762,0.631420,0.409524,70.475000,2010,1105,20111105,0.422939,0.283019,0.356140,0.238994,0.710843,0.315650,71.070000
4,20101106,0.444602,0.348936,0.406706,0.337017,0.732892,0.403471,67.492308,2010,1106,20111106,0.440000,0.348259,0.356952,0.293532,0.682226,0.405088,64.853333


In [39]:

corrs = []
cols = ['FG%','FGM3%','A_FG%','A_FGM3%','DRR','ORR','Pace']

for col in cols:
    ncol = 'N'+col
    corrs.append(np.round(gb[col].corr(gb[ncol]),2))


pdf = pd.DataFrame([corrs],columns=cols)

,FG%,FGM3%,A_FG%,A_FGM3%,DRR,ORR,Pace
0,0.32,0.22,0.34,0.07,0.42,0.53,0.5


In [50]:

seasons = list(range(2010,2019))
all_df = None
for season in seasons:
    season_path = "PL_Season_Stats_{}.csv".format(season)
    season = pd.read_csv(output+season_path)
    if all_df is not None:
        all_df = pd.concat([all_df,season],axis=0)
    else:
        all_df = season

all_df.to_csv("PL_Returning.csv",index=None)

